In [151]:
import seaborn
import pandas as pd
import itertools
import numpy as np
from matplotlib import pyplot as plt
import re
import color
import warnings

ModuleNotFoundError: No module named 'color'

In [184]:
def four_element_parser(line):
    
    count = int(line.split()[0])
    origin_x, origin_y, origin_z = [float(each) for each in line.split()[1:4]]
    
    return count, origin_x, origin_y, origin_z

def atom_parser(line):
    
    count = int(line.split()[0])
    origin_x, origin_y, origin_z = [float(each) for each in line.split()[2:5]]
    
    return count, origin_x, origin_y, origin_z


def compute_density(density, coordinate, radius, sphere=False):
    xC,yC,zC=coordinate
    
    densitylist = []
    for x in range(-radius+xC, radius+1+xC):
        for y in range(-radius+yC, radius+1+yC):
            for z in range(-radius+zC, radius+1+zC):
                if not sphere:
                    densitylist.append(density[x,y,z])
                elif ((x-xC)**2 + (y-yC)**2 + (z-zC)**2 <= radius*radius):
                    densitylist.append(density[x,y,z])
                    
    return densitylist

def read_log(filename, index, inputlist):
    log_density_list = []
    #dirty trick to obtain the atom's Fermi contact table
    with open(filename) as fptr:
        fermi_flag=False
        i=0
        for line in fptr:
            if ("Fermi" not in line) and (fermi_flag is False):
                continue
            elif (fermi_flag is False):
                fermi_flag=True
                fptr.readline()
                continue
                
                
            if (fermi_flag is True) and (len(line.split())==6):
                if int(line.split()[0]) in inputlist:
                    log_density_list.append(float(line.split()[index]))
            else:
                fermi_flag = False
    
    return np.array(log_density_list).flatten()

def extract_g(logfile):
    with open(logfile) as fptr:
        for line in fptr:
            if "g_RMC + g_DC" in line:
                #print(float(fptr.readline().split('=')[1].replace('D', 'E')))
                #print(float(fptr.readline().split('=')[3].replace('D', 'E')))
                #print(float(fptr.readline().split()[5].replace('D', 'E')))
                g1=float(list(filter(None,re.split('=| |-|\+', fptr.readline())))[1].replace('D',''))
                g2=float(list(filter(None,re.split('=| |-|\+', fptr.readline())))[4].replace('D',''))
                g3=float(list(filter(None,re.split('=| |-|\+', fptr.readline())))[7].replace('D',''))
                
    
    return (g1+g2+g3)*10 / 3

def extract_sigmaorb(logfile, nuclear='H'):
    sigmaorb = []
    
    with open(logfile) as fptr:
        for line in fptr:
            if (nuclear+"    Isotropic") in line:
                sigmaorb.append(float(line.split()[4]))
                
    
    return np.array(sigmaorb)


In [185]:
extract_sigmaorb("data/n4py_S1_nmr.gjf.log")

array([25.5332, 23.3829, 22.9875, 23.4631, 21.9429, 26.8034, 26.4706,
       23.9608, 23.1693, 23.4451, 21.4716, 23.3829, 22.9875, 23.4631,
       21.9429, 26.8034, 26.4706, 23.9608, 23.1693, 23.4451, 21.4716])

In [26]:
from abc import ABC, abstractmethod
class Abstract_cube(ABC):
    
    @abstractmethod
    def _process_cube(self):
        pass
    
    @abstractmethod
    def get_cube(self):
        pass
    
    @abstractmethod
    def get_direction(self):
        pass
    
    @abstractmethod
    def get_origin(self):
        pass
    
    @abstractmethod
    def get_atom_list(self):
        pass

In [168]:
class Spin_cube(Abstract_cube):
    def __init__(self, filename):
        self.filename = filename
        self._process_cube()
        self.density_list = None
    '''
    Dirty work of reading the cube file
    '''
    def _process_cube(self):
        with open(self.filename) as fptr:
            filecontent = fptr.readlines()

        #obtain constants
        title = " ".join(filecontent[0:2])
        atom_count, origin_x, origin_y, origin_z = four_element_parser(filecontent[2])

        voxel_X_count, X_d1, X_d2, X_d3 = four_element_parser(filecontent[3])
        voxel_Y_count, Y_d1, Y_d2, Y_d3 = four_element_parser(filecontent[4])
        voxel_Z_count, Z_d1, Z_d2, Z_d3 = four_element_parser(filecontent[5])

        ## tuple format: (atomic number, x, y, z)
        atom_list = []
        for atom_line in filecontent[6:6+atom_count]:
            atom_list.append(atom_parser(atom_line))


        ## parsing density
        LINEBREAK= int(voxel_Z_count / 6) + 1
        print(LINEBREAK)


        density = []
        newline = []
        for i,line in enumerate(filecontent[6+atom_count:]):
            newline += map(float, line.split())
            if (i % LINEBREAK == LINEBREAK-1):
                density.append(newline)
                newline = []

        spatial_density = []
        newline = []

        for i,Z_line in enumerate(density):
            newline.append(Z_line)

            if (i % voxel_Y_count == voxel_Y_count-1):
                spatial_density.append(newline)
                newline = []

        origin = (origin_x, origin_y, origin_z)
        Xd = (X_d1, X_d2, X_d3)
        Yd = (Y_d1, Y_d2, Y_d3)
        Zd = (Z_d1, Z_d2, Z_d3)

        self.origin=origin
        self.direction = (Xd, Yd, Zd)
        self.atom_list = atom_list
        self.density = np.array(spatial_density)
        

    def get_cube(self):
        return self.density
    

    def get_direction(self):
        return self.direction
    

    def get_origin(self):
        return self.origin
    
    
    def get_atom_list(self):
        return self.atom_list
    
    

In [231]:
class NMR_cube(Spin_cube):
    
    def _obtain_target(self, atomic_number, atoms):
        target_atoms=[]
        #order number of target atoms, start from 1

        for i, each in enumerate(atoms):
            if (each[0] == atomic_number):
                target_atoms.append(i+1)

        return target_atoms
    
    def output_spindensity(self, nuclear, radius, sphere_flag=True):
        self._nuclear = nuclear
        self._radius = radius
        H_list=self._obtain_target(atomic_number=nuclear, atoms=self.atom_list)
        density_list = []
        for each in H_list:
            atom_num, x, y, z = self.atom_list[each-1]
            relative_xyz = np.linalg.solve(np.array(self.direction), np.subtract((x,y,z), self.origin))

            density_list.append(compute_density(self.density, 
                                                (int(relative_xyz[0]), int(relative_xyz[1]), int(relative_xyz[2])), 
                                                     radius=radius, sphere=sphere_flag))
        self.density_list = density_list
        return density_list
    
    def get_mean_density(self):
        if self.density_list is None:
            raise Exception("No density calculated")
        else:
            return np.mean(self.density_list, axis=1)
    
    def get_std_density(self):
        if self.density_list is None:
            raise Exception("No density calculated")
        else:
            return np.std(self.density_list, axis=1)
    
    
    def compute_pNMRshift(self, nuclear, radius, temp=298, S=1, logfile=None, sphere_flag=True):
        if nuclear not in [1]:
            print("Only proton is allowed in this prediction")
            raise NotImplemented
        
        
        if (self.density_list is None) or (self._nuclear != nuclear) or (self._radius != radius):
            self.output_spindensity(nuclear, radius, sphere_flag)
        
        
        if logfile is None:
            import warnings
            warnings.warn("Log file not found, default value used", Warning)
            warnings.warn("Only paramagnetic shift is calculated", Warning)

            print("Log file is needed for computing")
            g_ave = 2.003

        else:
            g_ave = extract_g(logfile=logfile)
            
        
        if nuclear not in [1]:
            import warnings
            warnings.warn("You need to give delta_TMS for this nucleus!!!", Warning)
                
        deltaTMS=31.08 #ppm
        PI=3.14159265359 #
        gammaI=267.5
        muB=9.274*(10**(-1))
        A=10**6
        k=1.3806
        temp=temp
        S=S
            
        #compute the constant to link density with the A(MHz) and chemical shift(ppm)
        constant = (-2 * PI / gammaI) * (g_ave) * muB * A * (S+1)/(3*k*temp)
        #unit conversion from spin density to A(MHz)
        denToMHz = 2230.36 * 1.10
            
        
        density_list = self.output_spindensity(nuclear, radius, sphere_flag)
        mean_density = self.get_mean_density()
        std_density = self.get_std_density()
            
        mean_shift = mean_density * denToMHz * constant
        std_shift = -std_density * denToMHz * constant
        
        if logfile is None:
            return mean_shift, std_shift
        else:
            sigmaorb = extract_sigmaorb(logfile=logfile, nuclear='H')
            assert sigmaorb.shape == mean_shift.shape
            
            return deltaTMS-(sigmaorb+mean_shift), std_shift
            
            
        
        
        
        
        

In [232]:
project_name="data/Fe_MePy2tacn_CH3CN_OTf_2_nmr"

spin_cube = NMR_cube(project_name + ".cube")


22


In [233]:
##proton list for n4py
# H_list=[5,8,10,12,14,17,18,21,23,25,27,31,33,35,37,40,41,44,46,48,50]


# chk_density = np.array(spin_cube.output_spindensity(nuclear=1, radius=1))
# log_density =read_log(project_name + ".gjf.log", index=2, inputlist=H_list)
# # chk_density.shape
# plt.gcf().set_size_inches(10, 10)

# plt.errorbar(log_density, np.mean(chk_density, axis=1), yerr=np.std(chk_density,axis=1), 
#              ls='none', marker='s', markersize=3, elinewidth=2)
# plt.xlabel("log file density")
# plt.ylabel("averaged density")
mean, std = spin_cube.compute_pNMRshift(nuclear=1, radius=1, temp=298, S=1, logfile=project_name+".gjf.log", sphere_flag=True)


In [234]:
# from sklearn.linear_model import LinearRegression

# reg = LinearRegression().fit(mean.reshape(-1,1), log_density)
# reg.coef_


ValueError: Found input variables with inconsistent numbers of samples: [27, 21]

In [235]:
np.savetxt("output.csv", np.array([mean,std]).T, delimiter=',', fmt='%.5f')